In [1]:
import pandas as pd
import numpy as np

In [2]:
seg_df = pd.read_csv('DATA SET FOR RFM-VALUE BASED SEGMENTATION.csv',sep=',')

In [3]:
seg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22602 entries, 0 to 22601
Data columns (total 9 columns):
cust_id                 22602 non-null int64
No_of_days_frm_L_pur    22602 non-null int64
total_sale              22602 non-null float64
No_baskets              22602 non-null int64
item_cnt                22602 non-null int64
no_SKUs                 22602 non-null int64
no_Categories           22602 non-null int64
avg_no_prds_w_ret       22602 non-null float64
Flag_Multi_cat_p2       22602 non-null int64
dtypes: float64(2), int64(7)
memory usage: 1.6 MB


In [4]:
seg_df.head()

,cust_id,No_of_days_frm_L_pur,total_sale,No_baskets,item_cnt,no_SKUs,no_Categories,avg_no_prds_w_ret,Flag_Multi_cat_p2
0,347,3,444.00,4,47,25,5,11.75000,0
1,585,320,509871.48,27,19322,531,68,715.62963,0
2,885,115,2284.59,4,247,130,17,61.75000,1
3,1670,3,8092.17,28,553,306,114,19.75000,1
4,2188,428,200.50,1,51,14,2,51.00000,0


In [5]:
seg_df.columns

Index(['cust_id', 'No_of_days_frm_L_pur', 'total_sale', 'No_baskets',
       'item_cnt', 'no_SKUs', 'no_Categories', 'avg_no_prds_w_ret',
       'Flag_Multi_cat_p2'],
      dtype='object')

In [6]:
seg_df.rename(columns={'No_of_days_frm_L_pur': 'recency', 
                         'No_baskets': 'frequency', 
                         'total_sale': 'monetary_value'}, inplace=True)

In [7]:
seg_df.columns

Index(['cust_id', 'recency', 'monetary_value', 'frequency', 'item_cnt',
       'no_SKUs', 'no_Categories', 'avg_no_prds_w_ret', 'Flag_Multi_cat_p2'],
      dtype='object')

# Value based segmentation

In [15]:
# creating Deciles
seg_df['decile'] = pd.qcut(seg_df['monetary_value'], 5, labels=False)

In [16]:
seg_df.head(5)

,cust_id,recency,monetary_value,frequency,item_cnt,no_SKUs,no_Categories,avg_no_prds_w_ret,Flag_Multi_cat_p2,decile
0,347,3,444.00,4,47,25,5,11.75000,0,2
1,585,320,509871.48,27,19322,531,68,715.62963,0,4
2,885,115,2284.59,4,247,130,17,61.75000,1,4
3,1670,3,8092.17,28,553,306,114,19.75000,1,4
4,2188,428,200.50,1,51,14,2,51.00000,0,1


In [17]:
# Key performace variable selection
col_kpi=['monetary_value','recency','frequency','item_cnt','no_SKUs',
         'no_Categories','avg_no_prds_w_ret','Flag_Multi_cat_p2']

In [18]:
#Profiling
VB_Profiling=seg_df.groupby(['decile']).apply(lambda x: x[col_kpi].mean())

In [19]:
profiling_output = VB_Profiling.T

In [20]:
profiling_output

decile,0,1,2,3,4
monetary_value,88.519980,191.593081,402.966910,926.901805,4710.082013
recency,218.201902,179.186242,148.996901,89.390487,56.121876
frequency,1.019018,1.439505,2.574812,5.096903,13.905109
item_cnt,8.104379,14.796505,30.309208,65.232080,246.110595
no_SKUs,4.335913,7.387746,14.386454,32.706637,97.480646
no_Categories,1.936311,2.976111,5.637008,12.313053,36.406768
avg_no_prds_w_ret,7.966689,10.592656,12.895706,13.794329,20.027505
Flag_Multi_cat_p2,0.225343,0.371157,0.516158,0.749336,0.856005


In [14]:
profiling_output.to_csv('profiling_output.csv')

# RFM SEGMENTATION

In [ ]:
#Method-1: Creating quartiles

#seg_df['R_Quartile'] = pd.qcut(seg_df['recency'], 4, labels=False)
#seg_df['F_Quartile'] = pd.qcut(seg_df['recency'], 4, labels=False)
#seg_df['M_Quartile'] = pd.qcut(seg_df['recency'], 4, labels=False)

In [21]:
#Method-2: Creating quartiles

quantiles = seg_df[['frequency','monetary_value','recency' ]].quantile(q=[0.25,0.5,0.75])
quantiles

,frequency,monetary_value,recency
0.25,1.0,151.800,26.0
0.50,2.0,393.755,93.0
0.75,5.0,1117.725,225.0


In [22]:
quantiles = quantiles.to_dict()
quantiles

{'frequency': {0.25: 1.0, 0.5: 2.0, 0.75: 5.0},
 'monetary_value': {0.25: 151.8, 0.5: 393.755, 0.75: 1117.725},
 'recency': {0.25: 26.0, 0.5: 93.0, 0.75: 225.0}}

In [23]:
#We create two classes for the RFM segmentation since, being high recency is bad, while high frequency and monetary value is good.
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [24]:
seg_df['R_Quartile'] = seg_df['recency'].apply(RClass, args=('recency',quantiles,))
seg_df['F_Quartile'] = seg_df['frequency'].apply(FMClass, args=('frequency',quantiles,))
seg_df['M_Quartile'] = seg_df['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [25]:
seg_df.head()

,cust_id,recency,monetary_value,frequency,item_cnt,no_SKUs,no_Categories,avg_no_prds_w_ret,Flag_Multi_cat_p2,decile,R_Quartile,F_Quartile,M_Quartile
0,347,3,444.00,4,47,25,5,11.75000,0,2,1,2,2
1,585,320,509871.48,27,19322,531,68,715.62963,0,4,4,1,1
2,885,115,2284.59,4,247,130,17,61.75000,1,4,3,2,1
3,1670,3,8092.17,28,553,306,114,19.75000,1,4,1,1,1
4,2188,428,200.50,1,51,14,2,51.00000,0,1,4,4,3


In [26]:
seg_df1=seg_df.assign(R_seg=np.nan, FM_seg =np.nan)
seg_df1.R_seg[seg_df['R_Quartile']==4] = "3.Churn"
seg_df1.R_seg[seg_df['R_Quartile']==2] = "2.At Risk"
seg_df1.R_seg[seg_df['R_Quartile']==3] = "2.At Risk"
seg_df1.R_seg[seg_df['R_Quartile']==1] = "1.Active"

seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==1)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==1)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==2)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==2)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==4)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==1) & (seg_df['M_Quartile']==3)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==3)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==3)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==3)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==1) & (seg_df['M_Quartile']==4)] = "2.Gold"
seg_df1.FM_seg[(seg_df['F_Quartile']==2) & (seg_df['M_Quartile']==4)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==3) & (seg_df['M_Quartile']==4)] = "1.Silver"
seg_df1.FM_seg[(seg_df['F_Quartile']==4) & (seg_df['M_Quartile']==4)] = "0.Standard"

seg_df1.FM_seg.fillna("4.Premium", inplace=True)


C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\USER\Anaconda3\lib\site

In [27]:
seg_df1.head(5)

,cust_id,recency,monetary_value,frequency,item_cnt,no_SKUs,no_Categories,avg_no_prds_w_ret,Flag_Multi_cat_p2,decile,R_Quartile,F_Quartile,M_Quartile,R_seg,FM_seg
0,347,3,444.00,4,47,25,5,11.75000,0,2,1,2,2,1.Active,2.Gold
1,585,320,509871.48,27,19322,531,68,715.62963,0,4,4,1,1,3.Churn,4.Premium
2,885,115,2284.59,4,247,130,17,61.75000,1,4,3,2,1,2.At Risk,4.Premium
3,1670,3,8092.17,28,553,306,114,19.75000,1,4,1,1,1,1.Active,4.Premium
4,2188,428,200.50,1,51,14,2,51.00000,0,1,4,4,3,3.Churn,1.Silver


In [28]:
# Key performace variable selection
col_kpi=['monetary_value','recency','frequency','item_cnt','no_SKUs',
         'no_Categories','avg_no_prds_w_ret','Flag_Multi_cat_p2']

In [29]:
#Profiling
Profiling=seg_df1.groupby(['R_seg', 'FM_seg']).apply(lambda x: x[col_kpi].mean()).T

In [30]:
Profiling

R_seg                1.Active                                       \
FM_seg             0.Standard    1.Silver      2.Gold    4.Premium   
monetary_value     102.134684  242.719137  738.687137  3928.657833   
recency             11.890020   12.228013   11.742424    10.924857   
frequency            1.000000    1.596091    3.511164    14.798918   
item_cnt             8.152749   16.244300   44.181021   238.825669   
no_SKUs              4.472505    8.846906   21.535088   104.183950   
no_Categories        1.902240    3.434853    8.086124    39.227532   
avg_no_prds_w_ret    8.152749   10.463355   13.470282    15.814134   
Flag_Multi_cat_p2    0.535642    0.721498    0.912281     0.972047   

R_seg               2.At Risk                                       \
FM_seg             0.Standard    1.Silver      2.Gold    4.Premium   
monetary_value      98.560910  238.873325  780.129847  2366.371868   
recency            121.963967  114.298795  101.602462    81.002498   
frequency            1.000000    1.509237    3.247172     8.363835   
item_cnt             8.047656   16.590361   41.909182   137.563710   
no_SKUs              4.330105    8.208434   19.912176    55.733604   
no_Categories        1.813638    3.110442    7.337991    20.828232   
avg_no_prds_w_ret    8.047656   11.392303   13.988440    17.740587   
Flag_Multi_cat_p2    0.419217    0.546586    0.718563     0.811680   

R_seg                 3.Churn                                        
FM_seg             0.Standard    1.Silver       2.Gold    4.Premium  
monetary_value      94.664424  233.251813  1055.247251  3775.144343  
recency            351.511019  330.308901   326.814343   309.363029  
frequency            1.000000    1.479712     3.064542     4.786192  
item_cnt             8.594595   17.469895    37.080478   137.200445  
no_SKUs              4.600832    8.229712    14.170518    31.051225  
no_Categories        2.095218    3.328534     6.287649    11.233853  
avg_no_prds_w_ret    8.594595   11.961627    13.833625    24.139014  
Flag_Multi_cat_p2    0.000000    0.000000     0.000000     0.000000

In [33]:
Profiling.to_csv('Profiling_rfm.csv')